In [ ]:
import pandas as pd
import psycopg2
import dotenv
import os

dotenv.load_dotenv()

In [ ]:
with open('other_ids.txt', 'r') as file:
    ids = file.readlines()

ids = [id.strip() for id in ids]
df = pd.DataFrame(ids, columns=['id'])
df = df[1:]

print(len(df))

In [ ]:
conn = psycopg2.connect(host="ep-calm-star-76049617.us-west-2.retooldb.com", port = 5432, dbname = "retool", user = "retool", password = os.environ['RETOOL_PASSWORD'])
existing_ids = pd.read_sql_query('SELECT truck_id as id FROM cyber_workers', con=conn)
df = df[~df['id'].isin(existing_ids['id'])]
print(len(df))

In [ ]:
to_allocate = pd.read_csv('emails/molika.csv')
allocated = pd.read_sql_query('SELECT email FROM cyber_workers', con=conn)

print(len(to_allocate))
to_allocate = to_allocate[~to_allocate['email'].isin(allocated['email'])]
print(len(to_allocate))

In [ ]:
node_id = int(input('Enter node id: '))

with conn.cursor() as cur:
    for i, row in to_allocate.iterrows():
        email = row['email']
        id = df.iloc[i]['id']
        cur.execute(f"INSERT INTO cyber_workers (email, truck_id, node_id) VALUES ('{email}', '{id}', {node_id})")
    conn.commit()